# Processing PRF #
Notebook walking through a 'full' processing of ATL03/ATL08 data and compare it to Airborne Lidar Scanning (ALS) data in reference to the Petawawa Research Forest (45.997, -77.428).  This notebook will read ATL03/ATL08 file, rebin to 30 m, and read, process, and rebin the ALS data to a comparable swath of reference data.

### Importing PhoREAL modules ###

Start by installing PhoREAL modules

In [1]:
# Import packages
import os
import numpy as np
import scipy.io as sio
import pandas as pd
import random

# Read specific packages from PhoREAL
from phoreal.reader import get_atl03_struct
from phoreal.reader import get_atl08_struct
from phoreal.reader import get_atl_alongtrack
from phoreal.binner import rebin_atl08
from phoreal.binner import rebin_truth
from phoreal.binner import match_truth_fields
from phoreal.io import getTruthFilePaths, getTruthHeaders
from phoreal.reference import reprojectHeaderData
from phoreal.reference import findMatchingTruthFiles
from phoreal.reference import loadLasFile
from phoreal.reference import make_buffer
from phoreal.ace import ace
from phoreal.CalVal import perfect_classifier
from phoreal.getMeasurementError import getMeasurementError

## Read ATL03/ATL08 Files ##


Speicfy the ATL03 file, ATl08 file, and groundtrack to be used.

In [2]:
atl03_file = '/mnt/bigtex/vol1/Data/ICESat-2/REL005/ATL03/PRF/ATL03_20200208094726_06670606_005_01.h5'
atl08_file = '/mnt/bigtex/vol1/Data/ICESat-2/REL005/ATL08/PRF/ATL08/ATL08_20200208094726_06670606_005_01.h5'
gt = 'gt2l'
epsg_code = '32618'

Read the ATL03 file.  We can also specify we want this file to be in UTM Zone 18N, so we can specify this by defining it as the EPSG code.

In [3]:
atl03 = get_atl03_struct(atl03_file, gt, atl08_file, epsg = epsg_code)

Trim the ATL03 file to the Area-of-Interest -- not required but will keep us from processing areas outside of PRF. (this is something that should be made into a object function)

In [4]:
# Filter atl03.df by size of reproject
atl03.df = atl03.df[atl03.df.lat_ph > 45.930351153400515]
atl03.df = atl03.df[atl03.df.lat_ph < 46.02060957038562]
atl03.df = atl03.df.reset_index()
atl03.df, atl03.rotationData = get_atl_alongtrack(atl03.df)

Read the ATL08 file.  This will read the ATL08 data at the `land_segment` subgroup, which is the data at the 100m or 5 20m geo-segments. Inputting the ATL03 PhoREAL object will match the alongtrack information between files.

In [5]:
atl08 = get_atl08_struct(atl08_file, gt, atl03) 

Not nessisary but we can trim this file based on the ATL03 extent.

In [6]:
atl08.df = atl08.df[atl08.df.latitude > np.min(atl03.df.lat_ph)]
atl08.df = atl08.df[atl08.df.latitude < np.max(atl03.df.lat_ph)]

## Rebinning ATL08 to 30m ##

Now we can rebin the ATL08 data at the 30m using the binner functionality.

In [7]:
res_field = 'alongtrack'
res = 30

# ATL08 30m
atl08_bin = rebin_atl08(atl03, atl08, gt, res, res_field)

subset_can_flag
surf_type
subset_te_flag
The first part...
0.037378549575805664


/home/ejg2736/miniconda3/envs/test38_5/lib/python3.8/site-packages/phoreal-3.31.0.3-py3.8.egg/phoreal/binner.py:585: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_filter.drop(df_filter.columns.difference([key_field,field]),
/home/ejg2736/miniconda3/envs/test38_5/lib/python3.8/site-packages/phoreal-3.31.0.3-py3.8.egg/phoreal/binner.py:585: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filter.drop(df_filter.columns.difference([key_field,field]),
/home/ejg2736/miniconda3/envs/test38_5/lib/python3.8/site-packages/phoreal-3.31.0.3-py3.8.egg/phoreal/binner.py:585: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_filter.drop(df_fi

/home/ejg2736/miniconda3/envs/test38_5/lib/python3.8/site-packages/phoreal-3.31.0.3-py3.8.egg/phoreal/binner.py:569: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_filter.drop(df_filter.columns.difference([key_field,field]),
/home/ejg2736/miniconda3/envs/test38_5/lib/python3.8/site-packages/phoreal-3.31.0.3-py3.8.egg/phoreal/binner.py:569: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filter.drop(df_filter.columns.difference([key_field,field]),
/home/ejg2736/miniconda3/envs/test38_5/lib/python3.8/site-packages/phoreal-3.31.0.3-py3.8.egg/phoreal/binner.py:569: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_filter.drop(df_fi

The big compute...
0.8276786804199219
The unpacking...
0.0014801025390625
The trad canopy cover...
0.0028603076934814453
The sub_bin_canopy_metrics...
0.013291597366333008
Slope calc...
0.21829509735107422
Vertical Height Bins...
1.6972088813781738
Photons above threshold...
0.09105491638183594
The last part...
0.0014889240264892578


/home/ejg2736/miniconda3/envs/test38_5/lib/python3.8/site-packages/phoreal-3.31.0.3-py3.8.egg/phoreal/binner.py:1058: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  bin_df['beamNum'] = atl03.beamNum
/home/ejg2736/miniconda3/envs/test38_5/lib/python3.8/site-packages/phoreal-3.31.0.3-py3.8.egg/phoreal/binner.py:1059: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  bin_df['beamStrength'] = atl03.beamStrength
/home/ejg2736/miniconda3/envs/test38_5/lib/python3.8/site-packages/phoreal-3.31.0.3-py3.8.egg/phoreal/binner.py:1060: Performanc

Print the columns just to take a look..

In [8]:
atl08_bin

,h_ind,alongtrack,crosstrack,time,easting,northing,asr,atlas_pa,beam_azimuth,beam_coelev,...,h_bin_10,h_bin_11,h_bin_12,n_photons_above_threshold,beamNum,beamStrength,year,month,day,epsg
0,0,39.796092,0.424001,0.005556,315724.829634,5.099052e+06,0.465797,0.011231,1.391275,1.559565,...,0.00,0.000000,0.0,28.0,4,weak,2020,02,08,32618
1,1,64.818956,0.404777,0.009092,315721.722525,5.099027e+06,0.458679,0.011231,1.391298,1.559565,...,0.00,0.166667,0.0,6.0,4,weak,2020,02,08,32618
2,2,89.841820,0.385552,0.012628,315718.615415,5.099003e+06,0.451561,0.011232,1.391321,1.559565,...,0.00,0.052632,0.0,19.0,4,weak,2020,02,08,32618
3,3,114.864683,0.366328,0.016165,315715.508305,5.098978e+06,0.444443,0.011232,1.391344,1.559565,...,0.00,0.000000,0.0,10.0,4,weak,2020,02,08,32618
4,4,139.887547,0.347104,0.019701,315712.401196,5.098953e+06,0.437325,0.011232,1.391367,1.559564,...,0.25,0.000000,0.0,4.0,4,weak,2020,02,08,32618
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
330,330,9907.943633,0.355759,1.393489,314492.039342,5.089261e+06,0.286128,0.011244,1.391374,1.559553,...,0.00,0.000000,0.0,10.0,4,weak,2020,02,08,32618
331,331,9941.288846,0.142359,1.398179,314488.085150,5.089228e+06,0.286128,0.011244,1.391356,1.559553,...,0.00,0.000000,0.0,14.0,4,weak,2020,02,08,32618
332,332,9974.634059,-0.071042,1.402868,314484.130959,5.089195e+06,0.286128,0.011243,1.391339,1.559553,...,0.00,0.000000,0.0,5.0,4,weak,2020,02,08,32618
333,333,10007.979271,-0.284442,1.407558,314480.176768,5.089162e+06,0.286128,0.011243,1.391321,1.559553,...,0.00,0.000000,0.0,5.0,4,weak,2020,02,08,32618


## Generating Reference Swath ##
The following walks through PhoREAL reference in how to generate a reference swath

### Generating Header File ###
First part is defining the extents of the ALS data in the given repo.  This allows later to help more quickly identify which ALS files a groundtrack intersects.  The header file created is written out as an ASCII text file and placed in the same folder as reference data so that the file would only need to be generated once the first time the file is examined.  All of the files are identified, and then iterated through to find their extents.

In [9]:
truthSwathDir = '/mnt/bigtex/vol2/Data/OpenData/PRF/2018spl_2959'
truthFileType = 'laz'
truthFilePaths = getTruthFilePaths(truthSwathDir, truthFileType, logFileID=False)

truthHeaderDF = getTruthHeaders(truthFilePaths, truthFileType, logFileID=False)

   Previous header file exists
   Stored header file is ahead of reference directory, using only reference files selected



### Reporject Header File###
We can now reproject the Truth Header DF to have the same projection as the ATL03/ATL08 files.

In [10]:
epsg_atl = 'epsg:32618'
truthHeaderNewDF = reprojectHeaderData(truthHeaderDF, epsg_atl)

/home/ejg2736/miniconda3/envs/test38_5/lib/python3.8/site-packages/phoreal-3.31.0.3-py3.8.egg/phoreal/reference.py:185: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  truthHeaderNewDF['xmin'][i] = xout[0]
/home/ejg2736/miniconda3/envs/test38_5/lib/python3.8/site-packages/phoreal-3.31.0.3-py3.8.egg/phoreal/reference.py:186: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  truthHeaderNewDF['xmax'][i] = xout[1]
/home/ejg2736/miniconda3/envs/test38_5/lib/python3.8/site-packages/phoreal-3.31.0.3-py3.8.egg/phoreal/reference.py:187: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See th

In [11]:
truthHeaderNewDF

,index,fileName,version,xmin,xmax,ymin,ymax,zmin,zmax,nPoints,epsg
0,142,/mnt/bigtex/vol2/Data/OpenData/PRF/2018spl_295...,LAS v1.4,302999.832461,303999.792608,5.091001e+06,5.092001e+06,163.22,259.10,31714902,epsg:32618
1,143,/mnt/bigtex/vol2/Data/OpenData/PRF/2018spl_295...,LAS v1.4,296999.831398,297999.791545,5.093001e+06,5.094001e+06,201.66,335.61,34233714,epsg:32618
2,144,/mnt/bigtex/vol2/Data/OpenData/PRF/2018spl_295...,LAS v1.4,302999.832483,303999.792630,5.090001e+06,5.091001e+06,182.48,284.22,35524604,epsg:32618
3,145,/mnt/bigtex/vol2/Data/OpenData/PRF/2018spl_295...,LAS v1.4,296999.831376,297999.791523,5.094001e+06,5.095001e+06,203.21,301.82,37974434,epsg:32618
4,146,/mnt/bigtex/vol2/Data/OpenData/PRF/2018spl_295...,LAS v1.4,302999.832438,303999.792585,5.092001e+06,5.093001e+06,178.31,267.47,36128262,epsg:32618
...,...,...,...,...,...,...,...,...,...,...,...
137,279,/mnt/bigtex/vol2/Data/OpenData/PRF/2018spl_295...,LAS v1.4,314999.834473,315999.794620,5.092001e+06,5.093001e+06,151.85,191.09,38741907,epsg:32618
138,280,/mnt/bigtex/vol2/Data/OpenData/PRF/2018spl_295...,LAS v1.4,314999.834450,315999.794596,5.093001e+06,5.094001e+06,144.30,196.18,37713454,epsg:32618
139,281,/mnt/bigtex/vol2/Data/OpenData/PRF/2018spl_295...,LAS v1.4,314999.834427,315999.794573,5.094001e+06,5.095001e+06,139.10,196.58,37506958,epsg:32618
140,282,/mnt/bigtex/vol2/Data/OpenData/PRF/2018spl_295...,LAS v1.4,314999.834404,315999.794550,5.095001e+06,5.096001e+06,129.01,192.52,36483826,epsg:32618


### ID Intersecting ALS Tiles ###
We can now identify which tiles are intersecting.  The buffer for now is recommended to be 25 m for processing with the geolocation offset calculation.  The reference swath will be trimmed to the beam width later.

In [12]:
buffer = 25
_, matchingTruthFileInds = findMatchingTruthFiles(truthHeaderNewDF, 
                                                   atl03.df,  atl03.rotationData, buffer)
matchingTruthFiles = np.array(truthFilePaths)[matchingTruthFileInds]

### Generate Reference Swath ###
Generate the Reference Swath from the identified intersected ALS tiles

Here ACE is also run before being loaded into the truth swath for each granule.



In [13]:
truth_swath = pd.DataFrame()

for i in range(0,len(matchingTruthFiles)):
    print(matchingTruthFiles[i])
    truth_df = loadLasFile(matchingTruthFiles[i], epsg_atl, atl03.rotationData, decimate_n = 5)
    truth_df['classification'] = ace(np.array(truth_df.easting), np.array(truth_df.northing), 
                 np.array(truth_df.z), np.array(truth_df.classification))
    truth_df = make_buffer(atl03, truth_df, buffer)
    truth_swath = truth_swath.append(truth_df)

/mnt/bigtex/vol2/Data/OpenData/PRF/2018spl_2959/1kmZ183140509102018L.laz
      *Reference file EPSG code does not match ICESat-2, reprojecting reference file...


/tmp/ipykernel_394513/3576172717.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  truth_swath = truth_swath.append(truth_df)


/mnt/bigtex/vol2/Data/OpenData/PRF/2018spl_2959/1kmZ183140509202018L.laz
      *Reference file EPSG code does not match ICESat-2, reprojecting reference file...


/tmp/ipykernel_394513/3576172717.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  truth_swath = truth_swath.append(truth_df)


/mnt/bigtex/vol2/Data/OpenData/PRF/2018spl_2959/1kmZ183140509302018L.laz
      *Reference file EPSG code does not match ICESat-2, reprojecting reference file...


...

/tmp/ipykernel_394513/3576172717.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  truth_swath = truth_swath.append(truth_df)


/mnt/bigtex/vol2/Data/OpenData/PRF/2018spl_2959/1kmZ183150509502018L.laz
      *Reference file EPSG code does not match ICESat-2, reprojecting reference file...


/tmp/ipykernel_394513/3576172717.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  truth_swath = truth_swath.append(truth_df)


/mnt/bigtex/vol2/Data/OpenData/PRF/2018spl_2959/1kmZ183150509602018L.laz
      *Reference file EPSG code does not match ICESat-2, reprojecting reference file...


/tmp/ipykernel_394513/3576172717.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  truth_swath = truth_swath.append(truth_df)


### Compute Geolocation Offset ###
Compute the alongtrack, crosstrack, and vertical errors.

In [14]:
atlCorrections = getMeasurementError(atl03, truth_swath)

   Ground Track Number: gt2l (Beam #4, Beam Strength: weak)

   Comparing Reference Heights to ICESat-2 Ellipsoidal (HAE) Heights
   Using Ground Truth for ICESat-2 Offset Computation

   Test Case 1 of 4
   Raster Resolution = 8 m
   Cross Track Offsets = [-24, 24] (Adjusted to align with 8 m resolution)
   Along Track Offsets = [-24, 24] (Adjusted to align with 8 m resolution)
   Gridding ICESat-2 Data at 8 m Resolution using Mean Values...
   Gridding Reference Data at 8 m Resolution using Mean Values...
   Finding Offsets with Minimum MAE...
   Cross-Track Correction = 0 m (Easting = 0.0 m)
   Along-Track Correction = 0 m (Northing = 0.0 m)
   Vertical Correction = 35.8 m
   MAE = 0.35 m
   RMSE = 0.56 m

   Test Case 2 of 4
   Raster Resolution = 4 m
   Cross Track Offsets = [-12, 12] (Adjusted to align with 4 m resolution)
   Along Track Offsets = [-12, 12] (Adjusted to align with 4 m resolution)
   Gridding ICESat-2 Data at 4 m Resolution using Mean Values...
   Gridding Referen

Once computed they can be applied to the alongtrack, crosstrack, and z of the truth swath.

In [15]:
truth_swath.alongtrack = truth_swath.alongtrack - atlCorrections.alongTrack 
truth_swath.crosstrack = truth_swath.crosstrack - atlCorrections.crossTrack 
truth_swath.z = truth_swath.z - atlCorrections.z 

Finally, the beam width filter is applied to the reference swath so it only includes data within the beam footprint.  Current research indicates a beam diameter of 11 m, so 5.5 m should be selected as the beam width.

In [16]:
# Apply beam width measurement
truth_swath = make_buffer(atl03, truth_swath, 5)

### Perfect Classifier ###
Once the reference data is done processing the Perfect Classifier could be run if desired.

In [17]:
measpc, measoc = perfect_classifier(atl03, truth_swath,ground = [2],canopy = [3,4,5], 
                                          unclassed = [1, 6, 7, 18], keepsize = True)

Run Perfect Classifier
    Building KDtree
    Querying KDtree


### Rebin Reference Swath ###
Finally, we can rebin the reference swath.  

In [18]:
truth_bin = rebin_truth(atl03, truth_swath, res, res_field)

# Currently bug in rebinning_truth where alongtrack information 
# does not match up with the rebinned ATL08 data, this method
# will force it to be the same.  
truth_bin = match_truth_fields(truth_bin, atl08_bin)   